# Coursework: Optimisation of a fantasy football team

The coursework is described in detail in the documentation provided on Moodle. This notebook contains some code for basic functions that read in the data file and define the solution/constraint checker that you must use to check your final solution.

As noted in the coursework, you don't have to use Python or DEAP to tackle this. However, the practicals have covered a lot of functionality that will be useful so you should find that the DEAP libraries provide a quick way to start and will save you some time in writing code.

## Important Information

If you use another language, then you should write out your solution to a csv file as a comma separated list of 0,1s (one value per row) indicating which players are included, and use the code provided in this notebook to read it in and check it. You report should include the screenshot of the  output from the function provided in this notebook, and *not* your own version of the function



# Data
The code below reads in the datafile and calculates the number of players available.  
Change the filepath to your local drive.

The file is sorted by player type. As I may check your solution **DO NOT** sort the file or alter it in any way as my code will expect to see it in this format.

Feel free to browse the file and analyse the data in any way you think might be useful

## Papers Read:
https://www.erpublication.org/published_paper/IJETR031352.pdf (American) Football Team Selection Using Genetic Algorithm<br>

In [24]:
import pandas as pd
import numpy as np
import array
import random
import matplotlib
import matplotlib.pyplot as plt
import deap


# import deap packages required
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import benchmarks
from deap.benchmarks import binary as bnry

In [28]:
# THIS FUNCTION READS THE DATA FILE CONTAINING THE INFORMATION RE EACH PLAYER

# read data
data = (pd.read_csv("clean-data.csv").reset_index(drop=True))

num_players = len(data.index)

print("num possible players is %s" % (num_players))

num possible players is 523


# Helpful data
The code below extracts some useful information from the data that will be useful to you when writing your program. In particular:

- a list containing the **points** per player:  e.g. points[i] refers to the **points** associated with player *i*
- a list containing the **cost** per player: e.g. cost[i] refers to the **cost** associated with player *i*
- a list **gk** which indicates which player is a *goal-keeper*. The list is the same length as the number of players. gk[i]=0 if player *i* is not a goal-keeper; gk[i]=1 if player *i* is a goal-keeper
- a list **mid** which indicates which player is a *midfielder*. The list is the same length as the number of players. mid[i]=0 if player *i* is not a mid-fielder; mid[i]=1 if player *i* is a midfielder
- a list **defe** which indicates which player is a *defender*. The list is the same length as the number of players. defe[i]=0 if player *i* is not a defender; defe[i]=1 if player *i* is a defender
- a list **stri** which indicates which player is a *striker*. The list is the same length as the number of players. stri[i]=0 if player *i* is not a striker; stri[i]=1 if player *i* is a striker

In [29]:
# HELPFUL DATA 
# these can be used for calculating points and costs and are also used in the constraint_checking function
points = data['Points'] 
cost = data['Cost']
    
# create lists with all elements initialised to 0
gk = np.zeros(num_players)
mid = np.zeros(num_players)
defe = np.zeros(num_players)
stri = np.zeros(num_players)

for i in range(num_players):
    if data['Position'][i] == 'GK':
        gk[i] = 1
    elif data['Position'][i] == 'DEF':
        defe[i] = 1
    elif data['Position'][i] == 'MID':
        mid[i] = 1
    elif data['Position'][i] == 'STR':
        stri[i]=1

        
## Range of the 10 low genes (excludes 11, since specific range)
min_gene_range =(466, 0,   0,   0,   180, 180, 180, 376, 0,   180)
max_gene_range =(522, 179, 179, 179, 375, 375, 375, 465, 375, 465)

## Add defender range of the 11th gene
DEF_MIN, DEF_MAX = 0, 179
STR_MIN, STR_MAX = 376, 465

mut_chance = 3/11

In [30]:
Highest_Value_Chromosome = [470, 163, 15, 1, 180, 187, 196, 376, 29, 185, 0]
Binary_Highest_Value = convert_to_bin(Highest_Value_Chromosome)
print(check_constraints(Binary_Highest_Value))

total broken constraints: 0
total points: 2104.0
total cost is 100.0
selected players are [0, 1, 15, 29, 163, 180, 185, 187, 196, 376, 470]
(0, 2104.0)


In [31]:
# Convert Integer representation to Binary representation
def convert_to_bin(individual):
    binary_individual = np.zeros(num_players)
    for gene in individual:
        binary_individual[gene] = 1
    return binary_individual

In [39]:
# check the constraints
# the function MUST be passed a list of length num_players in which each bit is set to 0 or 1

def check_constraints(individual):
     
    broken_constraints = 0

    # exactly 11 players
    c1 = np.sum(individual)
    if  c1 != 11:
        broken_constraints+=1
        #print("total players is %s " %(c1))
        
    #need cost <= 100"
    c2 = np.sum(np.multiply(cost, individual)) 
    if c2 > 100:
        broken_constraints+=1
        #print("cost is %s " %(c2))
    
    # need only 1 GK
    c3 = np.sum(np.multiply(gk, individual))
    if  c3 != 1:
        broken_constraints+=1
        #print("goalies is %s " %(c3))
    
    # need less than 3-5 DEF"
    c4 = np.sum(np.multiply(defe,individual))
    if  c4 > 5 or c4 < 3:
        broken_constraints+=1
        #print("DEFE is %s " %(c4))
            
    #need 3- 5 MID
    c5 = np.sum(np.multiply(mid,individual))
    if  c5 > 5 or c5 < 3: 
        broken_constraints+=1
        #print("MID is %s " %(c5))
        
    # need 1 -1 3 STR"
    c6 = np.sum(np.multiply(stri,individual))
    if c6 > 3 or c6 < 1: 
        broken_constraints+=1
        #print("STR is %s " %(c6))
        
    # get indices of players selected
    selectedPlayers = [idx for idx, element in enumerate(individual) if element==1]
    
    totalpoints = np.sum(np.multiply(points, individual))
        
    #print("total broken constraints: %s" %(broken_constraints))
    #print("total points: %s" %(totalpoints))
    #print("total cost is %s" %(c2))
    #print("selected players are %s" %(selectedPlayers))
    
    return broken_constraints, totalpoints

In [40]:
# Evaluation of the problem
def football_eval(int_individual):
    binary_individual = convert_to_bin(int_individual) # Convert from int chromosome to binary chromosome
    # Since string is converted to binary, if multiple values are the same, it will convert with broken constraint
    
    broken, total_points = check_constraints(binary_individual) # returns #-broken-constraints, #-points-scored)
    
    for i in range(broken): # reduce broken constraints from the returned points
        total_points *= 0.6 # Multiply for each broken constraint
        
    return (total_points), # STOP.. COMMA TIME? - https://youtu.be/otCpCn0l4Wo

In [41]:
def num_of_broken_constraints(int_individual): # Returns the number of broken constraints (used for reporting)
    binary_individual = convert_to_bin(int_individual)
    broken, totalPoints = check_constraints(binary_individual)
    return broken

In [42]:
# Basically mutateUniformInt but custom written to deal with problem specific ranges
## Ordering is GK, DEF, DEF, DEF, MID, MID, MID, STR, DEF&MID, MID&STR, DEF&STR
def custom_mutate(individual):
    
    # Loop through first 10 genes
    for i in range(10):
        if np.random.rand() <= mut_chance:
            change = np.random.randint(low=-10, high=10)
            if (individual[i] + change) > max_gene_range[i]:
                individual[i] = max_gene_range[i]
            elif (individual[i] + change < min_gene_range[i]):
                individual[i] = min_gene_range[i]
            else:
                individual[i] += change
                
    # 11th Gene Custom Mutation - Wrap from Defender_Max -> Striker_Min and Striker_Min -> Defender_Max
    if np.random.rand() <= mut_chance:
        
        change = np.random.randint(low=-10, high=10)
        
        ## Only want to go from max_defender_rng to min_striker_rng and not from max_striker_rng to min_def_rng etc
        if (individual[10] <= DEF_MAX) and (individual[10] + change) > DEF_MAX:
            diff = (individual[10] + change) - DEF_MAX
            individual[10] = (STR_MIN - 1) + diff
        elif (individual[10] >= STR_MIN) and (individual[10] + change) < STR_MIN:
            diff = (individual[10] + change) - STR_MIN 
            individual[10] = (DEF_MAX + 1) + diff
        elif (individual[10] + change) > STR_MAX:
            individual[10] = STR_MAX
        elif (individual[10] + change) < DEF_MIN:
            individual[10] = DEF_MIN
        else:
            individual[10] += change    
    
    return (individual),

In [43]:
# Initialisation
def custom_initialisation(icls):
    
    # Initialise all as -1, as it checks to see if 0 or 1 are in array (as they are part of dset)
    arr_individual = icls(np.ones(11)*-1)
        
    # For first 10 values
    for i in range(10):
        num = np.random.randint(min_gene_range[i], max_gene_range[i])
        while(num in arr_individual):
            num = np.random.randint(min_gene_range[i], max_gene_range[i])
        arr_individual[i] = num
    
    # Generate value in upper or lower range for last val
    if (np.random.rand() >= 0.5):
        num = np.random.randint(STR_MIN, STR_MAX)
        while(num in arr_individual):
            num = np.random.randint(STR_MIN, STR_MAX)
    else:
        num = np.random.randint(DEF_MIN, DEF_MAX)
        while(num in arr_individual):
            num = np.random.randint(DEF_MIN, DEF_MAX)
    
    arr_individual[10] = num
    
    return arr_individual

# Solution and constraint checker function

You are free to represent an individiual in any way you wish. However, at the end of the evolutionary run, you *must* convert your solution to a list of length *num_players* in which each element is either 0 or 1. An element *i* should be set to 0 if player *i* is not included in the team, and to 1 if player *is* **is** included in the team.

You *must* call this function with your best solution and include a screen shot of the output in your report.

In [44]:
def init_cr_toolbox():
    creator.create("FitnessMax", base.Fitness, weights=(1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMax)

    toolbox = base.Toolbox() # Store yer damn tools in the toolbox
    
    ## Structure initializers ##
    # Specifying an 11-length permutation of integers as the players
    toolbox.register("individual", custom_initialisation, creator.Individual)

    #  Register a population of individuals
    toolbox.register("population", tools.initRepeat, list, toolbox.individual)
    
    toolbox.register("evaluate", football_eval)
    toolbox.register("mate", tools.cxTwoPoint)
    # CROSSOVER - cxPartialyMatched, cxUniformPartialyMatched DONT WORK
    toolbox.register("mutate", custom_mutate)
    toolbox.register("select", tools.selTournament, tournsize=6)
    #toolbox.register("select", tools.selNSGA2)
    
    return creator, toolbox

creator, toolbox = init_cr_toolbox() # Want to create and register inside method and then return to hide variables/complexity

## Come back to writing custom mutation operator
Mutate the individual - Checked both the cricket papers for mutation operators that were task appropriate
def mutate_player(individual):
return (individual), 

In [45]:
def main(pop_count=200, generations=200):
    
    # choose a population size: e.g. 200
    pop = toolbox.population(n=pop_count)
    
    # keep track of the single best solution found
    hof = tools.HallOfFame(1)
 
    # create a statistics object: we can log what ever statistics we want using this. We use the numpy Python library
    # to calculate the stats and label them with convenient labels
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)
    
    # run the algorithm: we need to tell it what parameters to use
    # cxpb = crossover probability; mutpb = mutation probability; ngen = number of iterations
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=1.0, mutpb=0.05, ngen=generations, 
                                   stats=stats, halloffame=hof, verbose=False)
    
    return pop, log, hof

In [46]:
## MULTI RUN EA ##
ITERATIONS = 50
avg_found = np.zeros(ITERATIONS)
avg_score = np.zeros(ITERATIONS) # report best scores after
brokens = np.zeros(ITERATIONS) # report broken constraints after
hofs = [] # report hofs after

pop_size = 200
total_gens = 200

for j in range(0, ITERATIONS):
    # run the main function 
    pop, log, hof = main(pop_size, total_gens)


    #  retrieve the best fitness found  that is stored at index 0 in the hof list
    best = hof[0].fitness.values[0] 


    # look in the logbook to see what generation this was found at

    max = log.select("max")  # max fitness per generation stored in log

    for i in range(total_gens):   # set to whatever ngen is
            fit = max[i]
            if fit == best:
                break
    
    avg_found[j] = i
    avg_score[j] = best
    
    new_hof = list(hof)
    hofs.append(new_hof)
    brokens[j] = num_of_broken_constraints(new_hof)

for i in range(ITERATIONS):
    print("{}".format(avg_score[i]))
    
for i in range(ITERATIONS):
    print("best solution is {}".format(hofs[i]))
    

print("Average score {} and score std dev {}. Max found in {} steps, and steps std dev {}".format(
        np.average(avg_score), round(np.std(avg_score), 2), np.average(avg_found), round(np.std(avg_found), 2)))

2037.0
2019.0
2042.0
1979.0
2055.0
1959.0
1982.0
2015.0
2001.0
1964.0
1969.0
2052.0
2011.0
2028.0
1999.0
2068.0
2026.0
2017.0
1983.0
2041.0
2030.0
2015.0
2038.0
2057.0
1976.0
2011.0
1993.0
2076.0
1973.0
2001.0
1992.0
2065.0
2032.0
1994.0
1987.0
1944.0
2004.0
2020.0
2095.0
1998.0
2003.0
2026.0
2007.0
2060.0
1954.0
2038.0
2022.0
1996.0
1998.0
2046.0
best solution is [[470, 61, 145, 28, 196, 181, 185, 376, 0, 183, 1]]
best solution is [[469, 28, 15, 163, 190, 180, 183, 376, 187, 211, 0]]
best solution is [[470, 1, 15, 9, 202, 198, 196, 376, 16, 203, 0]]
best solution is [[470, 9, 0, 10, 196, 185, 231, 376, 11, 183, 80]]
best solution is [[470, 10, 53, 0, 187, 180, 183, 376, 15, 185, 163]]
best solution is [[466, 15, 19, 0, 185, 320, 221, 383, 9, 183, 376]]
best solution is [[470, 1, 0, 43, 183, 196, 275, 382, 52, 376, 10]]
best solution is [[466, 1, 0, 28, 198, 267, 196, 376, 29, 187, 9]]
best solution is [[470, 15, 62, 52, 183, 187, 215, 376, 35, 180, 0]]
best solution is [[466, 52, 15, 

## Broken Constraints
cost is 100.4 <br>
total players is 10.0 <br>
DEFE is 2.0 <br>
selected players are [4, 20, 184, 185, 198, 211, 387, 433, 458, 510]<br>
broken - [510, 20, 4, 20, 211, 185, 184, 433, 198, 387, 458]<br>

MID is 6.0 <br>
selected players are [22, 104, 111, 185, 215, 243, 323, 328, 352, 418, 503]<br>

total players is 10.0 <br>
DEFE is 2.0 <br>
selected players are [2, 86, 182, 200, 204, 336, 377, 382, 419, 474]<br>
broken - [474, 86, 2, 86, 200, 204, 182, 382, 336, 377, 419]<br>

previous ~ 16 mins <br>
15:40 - 

## Broken Constraints After Init Changes

Sometimes, crossover means there are only 10 players in squad  <br>
Also, cost can be too high

And very occasionally too many players of 1 type (top is an example)


MID is 6.0 <br>
selected players are [22, 104, 111, 185, 215, 243, 323, 328, 352, 418, 503] <br>
broken - [503, 22, 104, 111, 352, 243, 323, 418, 328, 185, 215] <br>
cost is 83.2 <br>
total players is 10.0  <br>
selected players are [0, 126, 134, 178, 205, 228, 286, 324, 458, 497] <br>
broken - [497, 134, 126, 178, 228, 286, 205, 458, 178, 324, 0] <br>
cost is 78.4 <br>
total players is 10.0  <br>
selected players are [4, 34, 80, 120, 184, 218, 241, 313, 376, 517] <br>
broken - [517, 120, 4, 80, 241, 218, 184, 376, 34, 313, 4] <br>
cost is 88.70000000000002 <br>
total players is 10.0  <br>
selected players are [13, 17, 66, 91, 195, 235, 250, 307, 401, 466] <br>
broken - [466, 66, 91, 13, 235, 307, 195, 401, 17, 250, 66] <br>
cost is 82.30000000000001 <br>
total players is 10.0  <br>
selected players are [0, 4, 15, 85, 202, 209, 269, 389, 447, 485] <br>
broken - [485, 15, 4, 85, 202, 269, 269, 447, 0, 209, 389] <br>
cost is 84.10000000000001
total players is 10.0  <br>
selected players are [122, 123, 147, 187, 214, 264, 336, 376, 458, 474] <br>
broken - [474, 147, 122, 123, 214, 264, 187, 376, 336, 214, 458] <br>
cost is 78.9 <br>
total players is 10.0  <br>
selected players are [18, 54, 81, 146, 149, 188, 209, 269, 440, 496] <br>
broken - [496, 54, 146, 18, 188, 188, 209, 440, 81, 269, 149] <br>
cost is 74.5 <br>
total players is 10.0  <br>
selected players are [6, 87, 167, 200, 227, 231, 274, 340, 463, 521] <br>
broken - [521, 6, 87, 167, 231, 274, 200, 463, 340, 227, 87] <br>
cost is 78.19999999999999 <br>
total players is 10.0  <br>
selected players are [27, 50, 166, 174, 182, 196, 226, 405, 447, 501] <br>
broken - [501, 27, 166, 174, 182, 196, 226, 447, 50, 405, 405] <br>
cost is 83.8 <br>

# Code to read in saved solution

**You only need this code if you have written your coursework using another language and need to check the saved solution. Otherwise, ignore this section**

The code below expects you have saved you solution to a csv file. The file should contain 523 rows, each of which has a single value  set to 0 or 1.


Once you have read in your data, you will need to:
- read in the data file with the player information
- run the code in the cell above to create the lists cost/points/gk/defe/mid/stri
- run the check_constraints function passing the "individual" read from your file using the code below

In [ ]:
import csv

with open('pathtofile/mysolution.csv') as csvfile:
    readCSV = csv.reader(csvfile, delimiter=',')
    individual = []
    for row in readCSV:
        value=row[0]
        individual.append(int(value))

print(individual)

# check length
num_players = len(individual.index)

if num_players != 523:
    print("the solution file does not contain the correct number of variables")
    


## Data Exploration

In [11]:
### Trying to explore the dataset

## Individual totals
print(sum(gk))
print(sum(mid))
print(sum(defe))
print(sum(stri))
print("")

## Aggregated total
tot_players = sum(gk) + sum(mid) + sum(defe) + sum(stri)
print(tot_players)
print("")

## Percentage breakdowns
print((sum(gk) / tot_players) * 100)
print((sum(mid) / tot_players) * 100)
print((sum(defe) / tot_players) * 100)
print((sum(stri) / tot_players) * 100)

57.0
196.0
180.0
90.0

523.0

10.89866156787763
37.476099426386234
34.41682600382409
17.208413001912046


## Links
https://en.wikipedia.org/wiki/Metaheuristic - Checking Metaheuristics


## Process for my Solution

- Pre-week 7 content:
    - Basic data exploration (in code and Excel) to understand dataset better, to allow better formulation of EA
    - Read the 2 cricket papers
    - Multiply points by (70%) penalty per score
    - Following the 11 chromosome permutation used in the cricket examples, with ints as player refs
    - Writing custom mutation to alter genes in 
    - Looking at perhaps the sorting of the permutation (ordering doesn't matter)
        - If swapping goalies for strikers e.g. in crossover can destroy the fitness of new solutions

- Post-week 7 content:
    - 